### `Step-3` : Image Text Filter
- After applying a resolution filter, I noticed that some images contained text, which could negatively affect the model's learning accuracy. Therefore, I applied a text filter to remove images containing text. After this step, I downloaded the image data folder from HDFS to the local disk.The following code lines were executed in the Ubuntu terminal to perform the download :
  
  - rm -rf /home/hduser/CA1/dangerous_animals

  - hdfs dfs -get /user1/dangerous_animals /home/hduser/CA1/
  
- When I manually reviewed the images, I realized that some did not represent animals or were duplicates. I manually removed these types of images to improve dataset quality.

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
import numpy as np
import os

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Image Text Filter") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# HDFS folder path
hdfs_path = "hdfs://localhost:9000/user1/dangerous_animals/scorpion/"

# Function to detect potential text in an image
def has_text(image_data):
    try:
        image_array = np.array(image_data)
        gray = np.mean(image_array, axis=-1)  # Convert to grayscale
        
        # Edge detection using gradient change
        gradient = np.abs(np.diff(gray, axis=1))
        edge_intensity = np.mean(gradient)
        
        # Threshold for text detection
        return edge_intensity > 15
    except Exception as e:
        print(f"Error processing image: {e}")
        return False

# Load images from HDFS
image_df = spark.read.format("image").load(hdfs_path)

# Delete images with text-like patterns
def delete_images_with_text():
    deleted_count = 0
    
    for row in image_df.collect():
        image_path = row['image']['origin']
        if has_text(row['image']['data']):
            os.system(f"hdfs dfs -rm {image_path}")
            print(f"Deleted: {image_path}")
            deleted_count += 1

    print(f"\nTotal images deleted: {deleted_count}")

# Start deletion
delete_images_with_text()


25/03/21 22:54:50 ERROR Executor: Exception in task 4.0 in stage 0.0 (TID 4)/ 8]
java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.sql.catalyst.expressions.codegen.BufferHolder.grow(BufferHolder.java:80)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.grow(UnsafeWriter.java:63)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.writeAlignedBytes(UnsafeWriter.java:180)
	at org.apache.spark.sql.catalyst.expressions.codegen.UnsafeWriter.write(UnsafeWriter.java:153)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.execution.datasources.FileFormat$$anon$1.$anonfun$apply$1(FileFormat.scala:156)
	at org.apache.spark.sql.execution.datasources.FileFormat$$anon$1$$Lambda$2595/955746662.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.next(FileScanRDD.scala:211)


25/03/21 22:54:52 ERROR Utils: Uncaught exception in thread Executor task launch worker for task 2.0 in stage 0.0 (TID 2)
java.lang.NullPointerException
	at org.apache.spark.scheduler.Task.$anonfun$run$3(Task.scala:144)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1509)
	at org.apache.spark.scheduler.Task.run(Task.scala:142)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$3(Executor.scala:554)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1529)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:557)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/pyspark/errors/exceptions/captured.py", line 169, in deco
    return f(*a, **kw)
           ^^^^^^^^^^^

ConnectionRefusedError: [Errno 111] Connection refused

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [2]:
spark.stop()
